In [1]:
%matplotlib tk

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os

# Collecting Corresponding Points from images
### we will use:
- ginput(n)
- numpy.save(path, var) - save variable into binary to path
- numpy.load(path) - load that saved variable

In [3]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

def pick_and_save_correspondences(stitching_img_path, base_img_path, n_corresp = 4, sample_name = 'temp' ):
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(23, 16))
# collecting correspondances
    # Load or create two example images
    img1 = mpimg.imread(stitching_img_path)
    img2 = mpimg.imread(base_img_path)

    ax1.imshow(img1)
    ax1.set_title("Image 1")

    ax2.imshow(img2)
    ax2.set_title("Image 2")

    plt.tight_layout()


    # Number of points to pick
    pts1 = []
    pts2 = []
    correspondances = []
    # Pick points alternately
    for i in range(n_corresp):
        # take point 1 at img1
        print(f"Select point {i+1} in Image 1")
        plt.sca(ax1)
        p1 = plt.ginput(1)[0]
        plt.plot(p1[0], p1[1], 'ro')
        pts1.append(p1)

        # take point 2 at img2
        print(f"Select corresponding point {i+1} in Image 2")
        plt.sca(ax2)
        p2 = plt.ginput(1)[0]
        plt.plot(p2[0], p2[1], 'go')
        pts2.append(p2)

        # push correspondance to list
        correspondances.append((p1, p2))

        # Draw lines showing correspondence across subplots
        # Convert axes coordinates to normalised figure coordinates
        fig.canvas.draw()
        transFigure = fig.transFigure.inverted()
        coord1 = ax1.transData.transform(p1)
        coord2 = ax2.transData.transform(p2)
        fig_coord1 = transFigure.transform(coord1)
        fig_coord2 = transFigure.transform(coord2)
        line = plt.Line2D((fig_coord1[0], fig_coord2[0]),
                        (fig_coord1[1], fig_coord2[1]),
                        transform=fig.transFigure,
                        color='yellow', linestyle='--')
        fig.lines.append(line)
        plt.draw()
    
    plt.close()

    # saving selected points
    output_dir = f'./homography/{sample_name}/samples'
    os.makedirs(output_dir, exist_ok=True)
    
    output_fullpath = output_dir + f'/{n_corresp}_corresp'
    np.save(output_fullpath, correspondances)


### Plotting funs used

>We have **figure coordinate system (display)** that contains subplots, etc. (`fig`).
>Every figure is like a **canvas**.

---

#### 1️⃣ `fig.transFigure.inverted()` vs `fig.transFigure`

* **Display (figure) coordinate system** can be:

1. **Normalized** `(0,0)` to `(1,1)`

   * Independent of the figure size → `fig.transFigure` (it's just a coordinate system)
   * **MAPping:** `FIG.NORMAL_COORD → FIG.PIXEL_COORD` → `fig.transFigure.transform(coords)`
   * **MAPping:** `FIG.PIXEL_COORD → FIG.NORMAL_COORD` → `fig.transFigure.inverted().transform(coords)`

2. **In pixels**

   * Shows actual pixels on the screen
   * Depends on the size of the figure window (canvas) → `fig.transFigure.inverted()` (it's just coord system for fig in pixels)

---

#### 2️⃣ `ax_i.transData.transform(x, y)`

1. `ax1.transData.transform(x, y)` maps **data points** `(x, y)` on subplot `ax1` to **pixels** in display coordinates (relative to the figure canvas).
2. Similarly, `ax2.transData` does the same for subplot `ax2`.

---

#### 3️⃣ Why do we map clicked points up until getting normalized figure coordinates?

* Because we **cannot use `fig.transFigure.inverted` as the transform coordinate system** for drawing lines.

---


In [9]:
# collecting correspondances
im1_path = './images/paris/paris_c.jpg'
im2_path = './images/paris/paris_b.jpg' # (BASE IMG)

#result_path = result_root_dir + os.path.basename(im1_path)
for sample_size in [5]:
    pick_and_save_correspondences(im1_path, im2_path, sample_size, sample_name=f'paris/H_cb')

Select point 1 in Image 1
Select corresponding point 1 in Image 2
Select point 2 in Image 1
Select corresponding point 2 in Image 2
Select point 3 in Image 1
Select corresponding point 3 in Image 2
Select point 4 in Image 1
Select corresponding point 4 in Image 2
Select point 5 in Image 1
Select corresponding point 5 in Image 2


In [4]:
sample_path = './homography/cmpe_building/left1_left2/samples/4_corresp.npy'

def homography_by_sample(sample_path):
    # load saved sample info
    points_data = np.load(sample_path)

    # compute homography
    points_im1 = points_data[:, 0]
    points_im2 = points_data[:, 1]

    return computeH(points_im1, points_im2)

def computeH(points_im1, points_im2):

    # constructing CALIBRATION MATRIX
    X1 = points_im1[:, 0]
    Y1 = points_im1[:, 1]

    ones = np.ones_like(X1)
    zeros = np.zeros_like(X1)

    X2 = points_im2[:, 0]
    Y2 = points_im2[:, 1]

    # last 3 columns of rows of type-1
    NX2X1 = -1 * X2 * X1
    NX2Y1 = -1 * X2 * Y1
    NX2 = -1 * X2

    # last 3 columns of other rows
    NY2X1 = -1 * Y2 * X1
    NY2Y1 = -1 * Y2 * Y1
    NY2 = -1 * Y2

    row1 = np.stack([X1, Y1, ones, zeros, zeros, zeros, NX2X1, NX2Y1, NX2], axis = 1)
    row2 = np.stack([zeros, zeros, zeros, X1, Y1, ones, NY2X1, NY2Y1, NY2], axis = 1)

    A_matrix = np.vstack([row1, row2])

    # compute svd for minimisation problem: h = argmin_h(Ah)
    U, S, Vt = np.linalg.svd(A_matrix)
    # solution is last row of V
    h = Vt[-1, :]
    H = h.reshape(3,3)

    # normalised solution to right-bottom corner element
    H = (H / H[2,2])

    return H

## 🌀 Warping Image

### 0) Why not Forward Warping?

We **don’t use forward warping** because it’s inefficient and produces artifacts.
Since real images contain **noise** and **picked correspondence points aren’t perfectly accurate**,
some points in `img1` may get mapped to the **same pixel** in `img2` plane by the homography $ H_{12} $.

As a result, when we compute

$
(x', y', 1)^T = H_{12} \cdot (x, y, 1)^T
$

the set of mapped pixels ${(x', y')}$ may **not cover every pixel** in the target image —
causing **holes** (unfilled regions) where no source pixel lands.

---

### 1) Backward Warping Idea

The idea is simple:
we **invert the mapping** and instead ask:

> “For each pixel (x′, y′) in the output image (img2 plane),
> from which location (x, y) in img1 should its color come?”

Mathematically:

$
\text{color}(x', y') = \text{color}*{\text{img1}}(H*{12}^{-1} \cdot (x', y', 1)^T)
$

So for every output pixel, we **trace it backward** through $ H_{12}^{-1} $
to find where it came from in `img1`.

---

### 1.1) Constructing All (x′, y′, 1) Points

To apply $ H_{12}^{-1} $ to **all output pixels at once**,
we create a grid of all pixel coordinates on the output (img2) plane.

We use **`np.meshgrid`** to generate all $(x′, y′)$ combinations:

Perfect — here’s a clear explanation (with math) of **how `np.meshgrid` builds the full coordinate matrix** used in backward warping 👇

---

## 🧩 Constructing the Coordinate Matrix with `np.meshgrid`

Let’s say your **output image** (the plane we warp onto) has size:

$
\text{height} = h_{out}, \quad \text{width} = w_{out}
$

So, pixel coordinates on this image are integer grid points:

$
x' \in [0, w_{out}-1], \quad y' \in [0, h_{out}-1]
$

---

### 1️⃣ Build coordinate grids

We use:

```python
xs, ys = np.meshgrid(np.arange(w_out), np.arange(h_out))
```

* `np.arange(w_out)` → `[0, 1, 2, ..., w_out-1]`  → **x-coordinates**
* `np.arange(h_out)` → `[0, 1, 2, ..., h_out-1]` → **y-coordinates**
* `meshgrid` repeats them to form **full 2D coordinate grids**:

Example (for `w_out=3`, `h_out=2`):

```python
xs =
[[0, 1, 2],
 [0, 1, 2]]

ys =
[[0, 0, 0],
 [1, 1, 1]]
```

That means:

* first row of `xs, ys` corresponds to y=0 (top row of image)
* second row corresponds to y=1, etc.
Together they define a mesh of coordinate pairs:

```scss
    (0,0) (1,0) (2,0)
    (0,1) (1,1) (2,1)
```

---

### 2️⃣ Flatten into column form

We flatten both to 1D arrays:

```python
x_flat = xs.ravel()
y_flat = ys.ravel()
```

Result:

$
x_{flat} = [0, 1, 2, 0, 1, 2]
$
$
y_{flat} = [0, 0, 0, 1, 1, 1]
$

Each pair ((x_i', y_i')) corresponds to **one pixel** in the output image.

---

### 3️⃣ Convert to homogeneous coordinates

Add a third coordinate = 1 for all points:

```python
ones = np.ones_like(x_flat)
```

Then **stack** them as rows (axis=0):

```python
p2_h = np.stack([x_flat, y_flat, ones], axis=0)
```

---

### 4️⃣ Resulting matrix shape

`p2_h` is a **3×N matrix**, where $ N = h_{out} \times w_{out} $.

$
p_2^h =
\begin{bmatrix}
x'_1 & x'_2 & \dots & x'_N \\
y'_1 & y'_2 & \dots & y'_N \\
1 & 1 & \dots & 1
\end{bmatrix}
$

Each column corresponds to **where img2 pixel comes from, if it was on plane img1**.
---

### 5️⃣ Why this form?

This form allows us to apply the inverse homography to *all pixels at once* using matrix multiplication:

$
p_1^h = H_{12}^{-1} \cdot p_2^h
$

which efficiently gives the mapped coordinates ((x, y)) in the source image for **every output pixel**.

---

---

## Interpolating and Color setting at once

### 1️⃣ Ideal mathematical model (no noise, integer coordinates)

If your homography were **perfect** and pixel-aligned, then for each pixel ((x',y')) on the **output image (img2 plane)**,
you’d find its corresponding pixel ((x,y)) on **img1 plane** as:

$
[x, y, 1]^T ;=; H^{-1} [x', y', 1]^T
$

and ideally, those ((x,y)) would be **integers** (exact pixel centers).
So, the color mapping is:

$
\text{color}*{\text{out}}(x',y') ;=; \text{color}*{\text{img1}}(x,y)
$

---

### 2️⃣ Real-world case (noise, non-integer mappings) - OUR CASE

To get the color, we interpolate between the four neighboring pixels around ((x,y)):

$
\text{color}_{\text{img2}}(x',y') = \text{color}_{\text{img1}}\big( \text{Interpolation}(H^{-1} [x', y', 1]^T) \big)
$

---

### 🔍 Intuitive explanation

* $H^{-1}$ tells **where this pixel on the output came from** on img1.
* That source coordinate may not be an integer pixel → so we **interpolate** color from nearby pixels in img1.
* That’s why we say “backward warping”:
  we go **back** from output → input to **sample** colors smoothly.

---

## Making Warped Part Visible — `warp_full`

1. Apply transform so all warped corners have positive coordinates:  
   $$
   H_{\text{adjusted}} = 
   \begin{bmatrix}
   1 & 0 & -x_{\min}\\
   0 & 1 & -y_{\min}\\
   0 & 0 & 1
   \end{bmatrix} H
   $$
   ensuring $ H_{\text{adjusted}}^{-1}(x_b, y_b, 1) = (x, y, 1) $ lies in image grid.  

2. Set output size to  
   $$
      W = \lceil x_{\max} - x_{\min} \rceil, \quad 
      H = \lceil y_{\max} - y_{\min} \rceil,
   $$
   so all backward-transformed points fit within the visible output.


In [5]:
def warp(image, homography, output_shape=None):
    h_in, w_in = image.shape[:2]

    if output_shape is None:
        # get sizes of image (it's img1)
        height = image.shape[0]
        width = image.shape[1]
    else:
        height, width = output_shape

    # get matrix of all points as columns
    x_axis = np.arange(width)
    y_axis = np.arange(height)

    # rows we will stack for the matrix
    x_mesh, y_mesh = np.meshgrid(x_axis, y_axis)
    ones = np.ones_like(np.ravel(x_mesh))
    
    # A is matrix of all points in homogeneos coord.sys for plane 2
    P2 = np.stack([np.ravel(x_mesh), np.ravel(y_mesh), ones], axis = 0)

    # now, we need to get colors of them by backward transform
    P21 = np.dot(np.linalg.inv(homography), P2 )
    P21 = P21 / P21[2,:]
    
    # interpolating with opencv function
    
    # 1. get coordinate maps
    x_map = P21[0, :].reshape(height, width).astype(np.float32)
    y_map = P21[1, :].reshape(height, width).astype(np.float32)

    # 2. using interpolation function opencv that also makes mapping of colors
    warped_image = cv2.remap(image, x_map, y_map ,interpolation = cv2.INTER_LINEAR, borderMode = cv2.BORDER_CONSTANT, borderValue=0)
    
    # return interpolated and remaped 
    return warped_image


def warp_full(image, homography):
    h, w = image.shape[:2]

    # Find corners and warp them
    corners = np.array([[0, 0, 1],
                        [w, 0, 1],
                        [w, h, 1],
                        [0, h, 1]]).T
    warped_corners = homography @ corners
    warped_corners /= warped_corners[2, :]
    xs, ys = warped_corners[0], warped_corners[1]

    # Translate original corners to BASE img. plane
    xmin, xmax = xs.min(), xs.max()
    ymin, ymax = ys.min(), ys.max()

    # Get Translation matrix, making adjusted Homography
    trans = np.array([[1, 0, -xmin],
                      [0, 1, -ymin],
                      [0, 0, 1]])
    
    # H_adj_inverse will now, make base coord. to map at coorners of original img.
    # So that in Backward Transform. we will not have points outside original image plane.
    H_adj = trans @ homography

    out_w, out_h = int(np.ceil(xmax - xmin)), int(np.ceil(ymax - ymin))
    return warp(image, H_adj, (out_h, out_w))

## Task Implementation

In [6]:
def mosaic(images, homographies):
    # Frist: we need to get translation vector for resulting image 
    # to have all coords poisitive to be visible
    
    # 1. get all corners warped
    all_warped_corners = []
    for image, homography in zip(images, homographies):
        h, w = image.shape[:2]

        # Find corners and warp them
        corners = np.array([[0, 0, 1],
                            [w, 0, 1],
                            [w, h, 1],
                            [0, h, 1]]).T
        
        warped_corners = homography @ corners
        warped_corners /= warped_corners[2, :]
        all_warped_corners.append(warped_corners)
    all_warped_corners = np.hstack(all_warped_corners)

    # getting global corners for total mosaic
    x_min = all_warped_corners[0].min()
    x_max = all_warped_corners[0].max()
    y_min = all_warped_corners[1].min()
    y_max = all_warped_corners[1].max()

    # getting global Translation Matrix for making total mosaic visible
    T = np.array(
        [[1, 0, -x_min],
         [0, 1, -y_min],
         [0, 0,   1   ]])
    
    # getting total img (canvas) size for output img
    w_out = int(np.ceil(x_max - x_min))
    h_out = int(np.ceil(y_max - y_min))

    # getting warped imgs.
    warped_imgs = [warp(img, T @ homography, output_shape = (h_out, w_out) ) 
                   for img, homography in zip(images, homographies)]
    return warped_imgs

def blend(images):
    # basic maximum intensity blending
    blended_mosaic = np.zeros_like(images[0], dtype = np.float32)
    for image in images:
        blended_mosaic = np.maximum(blended_mosaic, image)
    # clip any overflows
    return np.clip(blended_mosaic ,0 ,255).astype(np.uint8)

### Task 1 — Paris

Steps:  
- [ ] (1) Compute homography \(H\) using correspondences: **paris_a → paris_b (base)**.  
- [ ] (2) Warp and stitch **{paris_a, paris_b, paris_c}** onto the **paris_b** base frame.


In [ ]:
# 1. Stitching Paris Images, with paris_b as base.
base_path =  './tasks/1'


img_paris_a = mpimg.imread('./images/paris/paris_a.jpg')
img_paris_b = mpimg.imread('./images/paris/paris_b.jpg')
img_paris_c = mpimg.imread('./images/paris/paris_c.jpg')

# for all sample examples
for sample_size in [4, 5, 6, 8, 10, 12, 15]:
    result_save_path = base_path + f'/samples/{sample_size}_corresp/'
    
    # -- making one PANORAMIC img --
    H_ab = homography_by_sample(f'./homography/paris/H_ab/samples/{sample_size}_corresp.npy')
    H_cb = homography_by_sample(f'./homography/paris/H_cb/samples/{sample_size}_corresp.npy')
    blended_mosaic = blend(mosaic([img_paris_a, img_paris_b ,img_paris_c], [H_ab, np.eye(3), H_cb]))
    os.makedirs(result_save_path, exist_ok=True)
    mpimg.imsave(result_save_path + f'panorama.jpg', blended_mosaic)

## Task 2

### left-to-right

Let

$$
H_{A\to B}
$$

denote the homography that maps coordinates from image $ A  →  B $.

Let $ I_{3×3} $ be the 3×3 identity matrix.


---
<div style =  "text-align: center; font-weight: bold;">
 Since our goal is to warp all imgs to left_2 img plane
</div>


$$
H_{\text{iter}}^{(0)} = I_{3\times3}
$$

$$
H_{\text{iter}}^{(1)} = H_{\text{left}_1 \to \text{left}_2}
$$

$$
H_{\text{iter}}^{(2)} = H_{\text{left}_1 \to \text{left}_2} H_{\text{middle}\to\text{left}_1}
$$

$$
H_{\text{iter}}^{(3)} = H_{\text{left}_1\to\text{left}_2} H_{\text{middle}\to\text{left}_1} H_{\text{right}_1\to\text{middle}}
$$

$$
H_{\text{iter}}^{(4)} = H_{\text{left}_1\to\text{left}_2} H_{\text{middle}\to\text{left}_1} H_{\text{right}_1\to\text{middle}} H_{\text{right}_2\to\text{right}_1}
$$

In [10]:
# 2.1
base_path =  './tasks/2'


# Left To Right
task_name = 'left_to_right'
for sample_size in [4, 6, 8, 10, 12, 15]:
    for building_name in ['cmpe_building']:
        # getting base image
        base_img = mpimg.imread(f'./images/{building_name}/left_2.jpg')
        
        # initial homography - identity
        cumulative_homography = np.eye(3)
        homographies = [np.eye(3)]
        images = [base_img]
        # path to save results
        save_path = base_path + f'/{task_name}/{building_name}/samples/{sample_size}_corresp'
        
        # iterate over subsequent pairs
        for prev_img_name, stitching_img_name, output_img_name in [
            ['left_2.jpg', 'left_1.jpg', 'mosaic_1'],
            ['left_1.jpg', 'middle.jpg', 'mosaic_2'], # instability at 10
            ['middle.jpg', 'right_1.jpg', 'mosaic_3'],
            ['right_1.jpg', 'right_2.jpg', 'mosaic_final']]:

            # stitching {stitching_img} to left_2.jpg plane
            stitching_img = mpimg.imread(f'images/{building_name}/{stitching_img_name}')
            images.append(stitching_img)

            # getting homography we already have in homography folder
            sample_path = f'./homography/{building_name}/{stitching_img_name}->{prev_img_name}/samples/{sample_size}_corresp.npy'
            H_to_prev = homography_by_sample(sample_path)
            H_to_prev/=H_to_prev[2,2]
            # getting homography from current (stitching_img) to leftmost
            cumulative_homography = cumulative_homography @ H_to_prev
            cumulative_homography /= cumulative_homography[2,2]
            # add homography
            homographies.append(cumulative_homography)
            
            # get mosaic
            partial_mosaic = blend(mosaic(images, homographies))
            
            # warping {stitching_img} to leftmost img (left_2.jpg)
            warp_result = warp_full(stitching_img, cumulative_homography)
            os.makedirs(save_path, exist_ok=True)
            mpimg.imsave(save_path + f'/{output_img_name}.jpg', partial_mosaic)
            mpimg.imsave(save_path + f'/(warped){stitching_img_name}.jpg', warp_result)



## **Middle-Out**
<center>

### **Image Notation**

$$
\begin{aligned}
L_2 &\equiv \texttt{left\_2.jpg}, \\
L_1 &\equiv \texttt{left\_1.jpg}, \\
M   &\equiv \texttt{middle.jpg}, \\
R_1 &\equiv \texttt{right\_1.jpg}, \\
R_2 &\equiv \texttt{right\_2.jpg}.
\end{aligned}
$$

---

### **Given Pairwise Homographies**

$$
\begin{aligned}
H_{L_1 \to L_2}, \quad
H_{M \to L_1}, \quad
H_{R_1 \to M}, \quad
H_{R_2 \to R_1}.
\end{aligned}
$$

---

### **Goal (Middle-Out Stitching Strategy)**

1. Stitch $L_1$ and $R_1$ onto $M$ → obtain $(\text{mosaic}_1)$  
2. Stitch $L_2$ and $R_2$ onto $\text{mosaic}_1$ → obtain $(\text{mosaic}_{\text{final}})$

---

### **Required Homographies (to Middle Coordinate Frame)**

$$
\begin{aligned}
H_1 &= H_{L_1 \to M} = (H_{M \to L_1})^{-1}, \\[6pt]
H_2 &= H_{R_1 \to M}, \\[6pt]
H_3 &= H_{L_2 \to M} 
     = \left(H_{L_1 \to L_2} \, H_{M \to L_1}\right)^{-1} 
     = (H_{M \to L_2})^{-1}, \\[6pt]
H_4 &= H_{R_2 \to M} 
     = H_{R_1 \to M} \, H_{R_2 \to R_1}
\end{aligned}
$$

---

### **Mosaic Construction**

$$
\begin{aligned}
\text{mosaic}_1 
    &= \text{blend}\big(\text{mosaic}([L_1, M, R_1], [H_1, I, H_2])\big), \\[8pt]
\text{mosaic}_{\text{final}} 
    &= \text{blend}\big(\text{mosaic}([L_2, \text{mosaic}_1, R_2], [H_3, I, H_4])\big).
\end{aligned}
$$

</center>


In [ ]:
# 2.2

# Middle-out
base_path =  './tasks/2'

# Left To Right
task_name = 'middle_out'
# Middle-out
base_path = './tasks/2'
task_name = 'middle_out'

for building_name in ['cmpe_building', 'north_campus']:
    # load images
    img_l1 = mpimg.imread(f'./images/{building_name}/left_1.jpg')
    img_l2 = mpimg.imread(f'./images/{building_name}/left_2.jpg')
    img_m  = mpimg.imread(f'./images/{building_name}/middle.jpg')
    img_r1 = mpimg.imread(f'./images/{building_name}/right_1.jpg')
    img_r2 = mpimg.imread(f'./images/{building_name}/right_2.jpg')

    for sample_size in [12, 15]:
        print(f'at sample_size = {sample_size}')
        save_path = base_path + f'/{task_name}/{building_name}/samples/{sample_size}_corresp/'
        # compute homographies
        H1 = np.linalg.inv(homography_by_sample(f'./homography/{building_name}/middle.jpg->left_1.jpg/samples/{sample_size}_corresp.npy'))
        H2 = homography_by_sample(f'./homography/{building_name}/right_1.jpg->middle.jpg/samples/{sample_size}_corresp.npy')
        H3 = np.linalg.inv(
            homography_by_sample(f'./homography/{building_name}/left_1.jpg->left_2.jpg/samples/{sample_size}_corresp.npy') @
            homography_by_sample(f'./homography/{building_name}/middle.jpg->left_1.jpg/samples/{sample_size}_corresp.npy')
        )
        H4 = H2 @ homography_by_sample(f'./homography/{building_name}/right_2.jpg->right_1.jpg/samples/{sample_size}_corresp.npy')

        # normalize
        print("H1:", H1)
        print("H2:", H2)
        print("H3:", H3)
        print("H4:", H4)

        # mosaic and blend imgs
        mosaic_1 = blend(mosaic(images = [img_l1, img_m, img_r1], homographies = [H1, np.eye(3), H2]))
        mosaic_final = blend(mosaic(images = [img_l2, img_l1, img_m, img_r1, img_r2], homographies = [H3, H1, np.eye(3), H2, H4]))

        os.makedirs(save_path, exist_ok=True)
        mpimg.imsave(save_path + 'mosaic_1.jpg', mosaic_1)
        mpimg.imsave(save_path + 'mosaic_final.jpg', mosaic_final)

at sample_size = 12
H1: [[ 1.21325503e+00 -1.40598382e-01 -6.64471378e+02]
 [ 1.37548368e-01  1.06695124e+00 -1.21495164e+02]
 [ 2.93400902e-04 -7.94582765e-05  8.41249024e-01]]
H2: [[ 5.99424125e-01  1.04614609e-01  4.95460327e+02]
 [-1.17854479e-01  9.06883937e-01  4.93989215e+01]
 [-3.07230680e-04  7.08269677e-05  1.00000000e+00]]
H3: [[ 1.21712679e+00 -1.31839572e-01 -1.49828732e+03]
 [ 2.92558822e-01  1.21405358e+00 -3.60877409e+02]
 [ 7.70157043e-04 -4.04306144e-05  3.12293682e-01]]
H4: [[-9.59230142e-02 -6.17988414e-01  7.92332974e+02]
 [-1.87527619e-01  4.04367778e-01  1.08304646e+02]
 [-5.35354698e-04 -4.93309787e-04  8.63576013e-01]]


NameError: name 'blend' is not defined


## **first-out-then-middle**
<center>


### **Given Pairwise Homographies**

$$
H_{L_1 \to L_2} \\
H_{M \to L_1} \\
H_{R_1 \to M} \\
H_{R_2 \to R_1}
$$

---

### **Step 1 — Left Mosaic (base: (L_1))**

$$
H_{L_2 \to L_1} = (H_{L_1 \to L_2})^{-1}, \quad
\text{mosaic}*{\text{left}} = \text{blend}\big([L_1, L_2], [I, H*{L_2 \to L_1}]\big)
$$

---

### **Step 2 — Right Mosaic (base: (R_1))**

$$
H_{R_2 \to R_1} \text{ (given)}, \quad
\text{mosaic}*{\text{right}} = \text{blend}\big([R_1, R_2], [I, H*{R_2 \to R_1}]\big)
$$

---

### **Step 3 — Final Mosaic (base: (M))**

$$
\begin{aligned}
H_{L_1 \to M} &= (H_{M \to L_1})^{-1}, \\
H_{L_2 \to M} &= (H_{M \to L_1} , H_{L_1 \to L_2})^{-1}, \\
H_{R_1 \to M} &= H_{R_1 \to M}, \\
H_{R_2 \to M} &= H_{R_1 \to M} ,\\ H_{R_2 \to R_1}, \\
\text{mosaic}*{\text{final}} &= \text{blend}\big([\text{mosaic}*{\text{left}}, M, \text{mosaic}*{\text{right}}], [H*{L_1 \to M}, I, H_{R_1 \to M}]\big)
\end{aligned}
$$

---
</center>

In [7]:
# 2.3

base_path =  './tasks/2'


task_name = 'first-out-then-middle'

for building_name in ['north_campus']:
    # load images
    img_l1 = mpimg.imread(f'./images/{building_name}/left_1.jpg')
    img_l2 = mpimg.imread(f'./images/{building_name}/left_2.jpg')
    img_m  = mpimg.imread(f'./images/{building_name}/middle.jpg')
    img_r1 = mpimg.imread(f'./images/{building_name}/right_1.jpg')
    img_r2 = mpimg.imread(f'./images/{building_name}/right_2.jpg')

    for sample_size in [12]:
        print(f'at sample_size = {sample_size}')
        save_path = base_path + f'/{task_name}/{building_name}/samples/{sample_size}_corres/'
        # compute homographies
        H_l2l1 = np.linalg.inv(homography_by_sample(f'./homography/{building_name}/left_1.jpg->left_2.jpg/samples/{sample_size}_corresp.npy'))
        H_r2r1 = homography_by_sample(f'./homography/{building_name}/right_2.jpg->right_1.jpg/samples/{sample_size}_corresp.npy')
        H_l1m = np.linalg.inv(homography_by_sample(f'./homography/{building_name}/middle.jpg->left_1.jpg/samples/{sample_size}_corresp.npy'))
        H_r1m = homography_by_sample(f'./homography/{building_name}/right_1.jpg->middle.jpg/samples/{sample_size}_corresp.npy')

        # mosaic and blend imgs
        mosaic_left = blend(mosaic(images = [img_l1, img_l2], homographies = [np.eye(3), H_l2l1]))
        mosaic_right = blend(mosaic(images = [img_r1, img_r2], homographies = [np.eye(3), H_r2r1]))
        mosaic_final = blend(mosaic(images = [img_l2, img_l1, img_m, img_r1, img_r2], homographies = [ H_l1m @ H_l2l1, H_l1m, np.eye(3), H_r1m, H_r1m @ H_r2r1 ]))
        os.makedirs(save_path, exist_ok=True)
        #mpimg.imsave(save_path + 'mosaic_left.jpg', mosaic_left)
        #mpimg.imsave(save_path + 'mosaic_right.jpg', mosaic_right)
        mpimg.imsave(save_path + 'mosaic_final.jpg', mosaic_final)

at sample_size = 12


## Effects of Corresponding Points

### Experimenting over Paris Dataset

In [ ]:
# Noisy Corresp Points with Gausian

img_paris_a = mpimg.imread('./images/paris/paris_a.jpg')
img_paris_b = mpimg.imread('./images/paris/paris_b.jpg')
img_paris_c = mpimg.imread('./images/paris/paris_c.jpg')



def add_noise(points, sigma):
    noisy_points = points + np.random.normal(0, sigma, points.shape)
    return noisy_points

# now, we can't just get homography since it'll use points without noise
# get manually corresp points

for sigma in [1, 5, 10]:

    # for all simga examples
    result_save_path = f'./noisy/{sigma}_sigma/'

    # load saved sample info
    points_ab  = np.load('./homography/paris/H_ab/samples/15_corresp.npy')
    points_bb  = np.load('./homography/paris/H_bb/samples/15_corresp.npy')
    points_cb  = np.load('./homography/paris/H_cb/samples/15_corresp.npy')

    # compute homography AB
    points_im1 = points_ab[:, 0]
    points_im2 = points_ab[:, 1]
    
    points_im1_noisy = add_noise(points_im1, sigma)
    points_im2_noisy = add_noise(points_im2, sigma)

    H_ab_noisy = computeH(points_im1_noisy, points_im2_noisy)


    # compute homography BB
    points_im1 = points_bb[:, 0]
    points_im2 = points_bb[:, 1]
    
    points_im1_noisy = add_noise(points_im1, sigma)
    points_im2_noisy = add_noise(points_im2, sigma)

    H_bb_noisy = computeH(points_im1_noisy, points_im2_noisy)



    # compute homography CB
    points_im1 = points_cb[:, 0]
    points_im2 = points_cb[:, 1]
    
    points_im1_noisy = add_noise(points_im1, sigma)
    points_im2_noisy = add_noise(points_im2, sigma)

    H_cb_noisy = computeH(points_im1_noisy, points_im2_noisy)


    
    blended_mosaic = blend(mosaic([img_paris_a, img_paris_b ,img_paris_c], [H_ab_noisy, H_bb_noisy, H_cb_noisy]))
    os.makedirs(result_save_path, exist_ok=True)
    mpimg.imsave(result_save_path + f'panorama.jpg', blended_mosaic)


PermissionError: [Errno 13] Permission denied: '/noisy'